In [ ]:
# Preamble

def is_main_module():
    """
    Returns whether this notebook is the main module
    i.e. not being run from anotherr notebook
    """
    return __name__ == '__main__' and '__file__' not in globals()

SAVE_FIGURES = False
SAVE_FIGURE_DPI = 300
DEBUG_PRINT_PLOTS = True
SAVE_FIGURE_FORMAT = "PDF"  # PDF, SVG, PNG, for saved images

In [ ]:
#import libraries
import sys
# add the path to the Python module directory
sys.path.insert(0, '../Python')

import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd

import Constants
import Utils

import SphericalEarthModel as SEM
import Prf as PRF
import Nesz as NESZ
import Aasr as AASR
import Rasr as RASR

import planararrayantenna as PAA

In [ ]:
%matplotlib inline
%config Inlinebackend.figure_format = 'svg'

# AESA Beamforming Verification RADARSAT-2 Wide Fine Mode

Utilises 3 beams, F0W1, F0W2 and F0W3.

Swaths for the beams: 170km, 150km, 120km

Incidence Angle coverage: 20 - 45 degrees

Nominal range resolution: 5.2 m

Nominal azimuth resolution: 7.7 m

Antenna mounting angle (antenna normal to the nadir direction): 29.8 degrees

NESZ: -24 +- 2dB

Using a common RX antenna model for all renages and individual TX antenna models for the different beams.

Making use of SCORE

In [ ]:
# system parameters
Frequency                 = 5.405e9
Wavelength                = Constants.SPEED_OF_LIGHT / Frequency
Altitude                  = 798e3
TxPowerPeak               = 1650
PulseLen                  = 21e-6
Prf                       = 2600
TxDutyCycle               = PulseLen * Prf
AntennaOffset             = 29.8      # Antenna Mounting angle/Mid-Look angle in degrees
NoiseFigure               = 4.3       # TRM Noise figure in dB
Losses                    = 4.1       # System losses: Processing atmosphere, taper, degradation, etc. in dB
ProcessedAzimuthBandwidth = 900       # Doppler bandwidth in Hz
NumRxBeams                = 3
ChirpBandwidth            = 30e6     # pulse bandwidth

# System requirements
ReqNesz                   = -24       # Required NESZ in dB
ReqRasr                   = -20       # Required RASR in dB
ReqAasr                   = -20       # Required AASR in dB
ReqResolutionAz           = 7.7       # Required Azimuth resolution in meters   (cross-range) 
ReqResolutionRg           = 5.2       # Required Elevation resolution in meters (ground range)

TxPowerAvg                = TxPowerPeak * TxDutyCycle
MidSwathIncidenceAngle    = SEM.IncidenceAngle(Altitude, Utils.deg2rad(AntennaOffset))

print("Altitude: {:.2f} km".format(Altitude / 1000))
print("Wavelength: {:.3f} m".format(Wavelength))
print("Peak TX Power: {:.1f} W".format(TxPowerPeak))
print("Average TX Power: {:.1f} W".format(TxPowerAvg))
print("PRF: {:.1f} Hz".format(Prf))
print("Pulse length: {:.2f} us".format(PulseLen * 1e6))

## Geometry

In [ ]:
# geometry code
IncidenceAngleRange = np.arange(20, 45.1, 0.1) # Scene incidence angle range in degrees
IncidenceAngleRangeRad = IncidenceAngleRange * (np.pi/180)
print("Incidence angle ranges: {:.2f} - {:.2f} deg".format(IncidenceAngleRange.min(), IncidenceAngleRange.max()))

LookAngleRangeRad = SEM.LookAngle(Altitude, IncidenceAngleRangeRad)
LookAngleRange = LookAngleRangeRad * (180/np.pi)
print("Look angle ranges: {:.2f} - {:.2f} deg".format(LookAngleRange.min(), LookAngleRange.max()))

### Swath analysis

In [ ]:
# Get the ground swath width for the different look angles and TX beamwidths
Test_Beamwidths = np.arange(1, 61, 1)          # beamwidth ranges between 1 and 60 degrees
Test_LookAngles = np.arange(17, 55., 5)
Test_LookAngles = np.append(Test_LookAngles, 29.8)
Test_Altitudes = Altitude

Test_wgr = []
for la in Test_LookAngles:
    _, _wgr = SEM.SwathGroundSwathFromBeamwidth(Test_Altitudes, Utils.deg2rad(la), Utils.deg2rad(Test_Beamwidths))
    Test_wgr.append(_wgr)
Test_wgr = np.asarray(Test_wgr)

# Plot the ground swath widths vs beamwidths & look angles
fig, ax1 = plt.subplots(1,1,figsize = (10,4))
for i in range(Test_wgr.shape[0]):
    ax1.plot(Test_Beamwidths, Test_wgr[i,:] / 1000, label=r"Look Angle = %.2f deg" % (Test_LookAngles[i]))
ax1.legend()
ax1.set_title('Ground Swath Width vs Elevation Beamwidth \n Altitude = %d km' % (Test_Altitudes / 1000))
ax1.set_xlabel(r"Elevation Beamwidth (deg)")
ax1.set_ylabel(r"Ground swath Width (km)")
ax1.grid(linestyle='dotted')
ax1.set_ylim(10, 440)
ax1.set_xlim(0,30)
# beam 1
ax1.axvline(x=10.8, linestyle="dotted")
ax1.axhline(y=170, linestyle="dotted")
# beam 2
ax1.axvline(x=7.8, linestyle="dotted", color="tab:green")
ax1.axhline(y=150, linestyle="dotted", color="tab:green")
# beam 3
ax1.axvline(x=5, linestyle="dotted", color="tab:orange")
ax1.axhline(y=120, linestyle="dotted", color="tab:orange")

plt.tight_layout()
plt.show()

# Beam 1
SetTxBeamwidth1 = 10.8
sw, gw =  SEM.SwathGroundSwathFromBeamwidth(Test_Altitudes, Utils.deg2rad(17.69), Utils.deg2rad(SetTxBeamwidth1))
print("Swath1 = {:.2f} km and Ground swath1 = {:.2f} km @ Look angle = {} deg and Elevation BW = {} deg".format(sw/1000, gw/1000, 17.69, SetTxBeamwidth1))

# Beam 2
SetTxBeamwidth2 = 7.8
sw, gw =  SEM.SwathGroundSwathFromBeamwidth(Test_Altitudes, Utils.deg2rad(28.49), Utils.deg2rad(SetTxBeamwidth2))
print("Swath2 = {:.2f} km and Ground swath2 = {:.2f} km @ Look angle = {} deg and Elevation BW = {} deg".format(sw/1000, gw/1000, 28.49, SetTxBeamwidth2))

# Beam 3
SetTxBeamwidth3 = 5
sw, gw =  SEM.SwathGroundSwathFromBeamwidth(Test_Altitudes, Utils.deg2rad(36.29), Utils.deg2rad(SetTxBeamwidth3))
print("Swath3 = {:.2f} km and Ground swath3 = {:.2f} km @ Look angle = {} deg and Elevation BW = {} deg".format(sw/1000, gw/1000, 36.29, SetTxBeamwidth3))

In [ ]:
# Look angle and incidence angle calculations for beams
near_incidence_angle = Utils.deg2rad(20)
far_incidence_angle = Utils.deg2rad(45)

near_look_angle = SEM.LookAngle(Altitude, near_incidence_angle)
far_look_angle = SEM.LookAngle(Altitude, far_incidence_angle)

antenna_normal_incidence_angle = SEM.IncidenceAngle(Altitude, Utils.deg2rad(AntennaOffset))

print("Near range:\n\tIncidence Angle: {:.2f}\tLook Angle: {:.2f} deg".format(Utils.rad2deg(near_incidence_angle),
                                                                             Utils.rad2deg(near_look_angle)))
print("Far range:\n\tIncidence Angle: {:.2f}\tLook Angle: {:.2f} deg".format(Utils.rad2deg(far_incidence_angle),
                                                                             Utils.rad2deg(far_look_angle)))
print("Mid-swath Incidence Angle: {:.2f} deg".format(AntennaOffset))

## Antenna Physical Parameters

In [ ]:
# Physical Antenna code
antLength     = 15
antHeight     = 1.37
elNumElements = 32
azNumElements = 16
efficiency    = 0.886
desiredElvElemSpacing = 0.77 * Wavelength         # spacing from RADARSAT-2 datasheet
desiredAzElemSpacing = antLength / azNumElements  # spacing not specified in datasheet

In [ ]:
# display the physical antenna characteristics in a table format
pd.set_option("display.precision", 3)
df = pd.DataFrame (
    {
        "Parameter" : ['Length (Azimuth)', 'Width (Elevation)', 'Wavelength', 
                       'Elevation Element Spacing', 'Azimuth Element Spacing',
                       '# Azimuth Elements',
                       '# Elevation Elements',
                       'Average TX Power', 'PRF'],
        "Value" : [antLength, antHeight, Wavelength, 
                   desiredElvElemSpacing, desiredAzElemSpacing, 
                   azNumElements, 
                   elNumElements, 
                   TxPowerAvg, Prf],
        "Unit" : ['m', 'm', 'm', 
                  'm', 'm',
                  '',
                  '',
                  'W', 'Hz']
    }
)
titles = ['Parameter', 'Value', 'Unit']
df.reindex(columns=titles)

### Near Beam (F0W1)

In [ ]:
# initialise the TX antenna object
numTxElevElements = 6

radarsat2_F0W1_TxAnt = PAA.PlanarArrayAntenna(num_el_elems    = numTxElevElements, 
                                              num_az_elems    = azNumElements, 
                                              el_elem_spacing = desiredElvElemSpacing, 
                                              az_elem_spacing = desiredAzElemSpacing, 
                                              efficiency      = efficiency)

# swath covered by TX beam
near_beam_look_angle = 17.69
_, txGroundSwath = SEM.SwathGroundSwathFromBeamwidth(Altitude,
                                                     Utils.deg2rad(near_beam_look_angle), 
                                                     radarsat2_F0W1_TxAnt.elevation_beamwidth(Wavelength))
txGroundSwath_F0W1 = txGroundSwath

In [ ]:
# display the TX antenna characteristics in a table format
pd.set_option("display.precision", 3)
df = pd.DataFrame (
    {
        "Parameter" : ['# Azimuth Elements', 'Azimuth Beamwidth',
                       '# TX Elevation Elements', 'TX Elevation Beamwidth', 
                       'TX Gain', 'Ground Swath'],
        "Value" : [azNumElements, Utils.rad2deg(radarsat2_F0W1_TxAnt.azimuth_beamwidth(Wavelength)),
                   numTxElevElements, Utils.rad2deg(radarsat2_F0W1_TxAnt.elevation_beamwidth(Wavelength)),
                   10*np.log10(radarsat2_F0W1_TxAnt.gain(Wavelength)),
                   txGroundSwath /1000],
        "Unit" : ['','deg', 
                  '','deg', 
                  'dBi', 'km']
    }
)
titles = ['Parameter', 'Value', 'Unit']
df.reindex(columns=titles)

### Middle Beam (F0W2)

In [ ]:
# initialise the TX antenna object
numTxElevElements = 9

radarsat2_F0W2_TxAnt = PAA.PlanarArrayAntenna(num_el_elems    = numTxElevElements, 
                                              num_az_elems    = azNumElements, 
                                              el_elem_spacing = desiredElvElemSpacing, 
                                              az_elem_spacing = desiredAzElemSpacing, 
                                              efficiency      = efficiency)

# swath covered by TX beam
middle_beam_look_angle = 28.49
_, txGroundSwath = SEM.SwathGroundSwathFromBeamwidth(Altitude,
                                                     Utils.deg2rad(middle_beam_look_angle), 
                                                     radarsat2_F0W2_TxAnt.elevation_beamwidth(Wavelength))

In [ ]:
# display the TX antenna characteristics in a table format
pd.set_option("display.precision", 3)
df = pd.DataFrame (
    {
        "Parameter" : ['# Azimuth Elements', 'Azimuth Beamwidth',
                       '# TX Elevation Elements', 'TX Elevation Beamwidth', 
                       'TX Gain', 'Ground Swath'],
        "Value" : [azNumElements, Utils.rad2deg(radarsat2_F0W2_TxAnt.azimuth_beamwidth(Wavelength)),
                   numTxElevElements, Utils.rad2deg(radarsat2_F0W2_TxAnt.elevation_beamwidth(Wavelength)),
                   10*np.log10(radarsat2_F0W2_TxAnt.gain(Wavelength)),
                   txGroundSwath /1000],
        "Unit" : ['','deg', 
                  '','deg', 
                  'dBi', 'km']
    }
)
titles = ['Parameter', 'Value', 'Unit']
df.reindex(columns=titles)

### Far Beam (F0W3)

In [ ]:
# initialise the TX antenna object
numTxElevElements = 13

radarsat2_F0W3_TxAnt = PAA.PlanarArrayAntenna(num_el_elems    = numTxElevElements, 
                                              num_az_elems    = azNumElements, 
                                              el_elem_spacing = desiredElvElemSpacing, 
                                              az_elem_spacing = desiredAzElemSpacing, 
                                              efficiency      = efficiency)

# swath covered by TX beam
far_beam_look_angle = 36.29
_, txGroundSwath = SEM.SwathGroundSwathFromBeamwidth(Altitude,
                                                     Utils.deg2rad(far_beam_look_angle), 
                                                     radarsat2_F0W3_TxAnt.elevation_beamwidth(Wavelength))

In [ ]:
# display the TX antenna characteristics in a table format
pd.set_option("display.precision", 3)
df = pd.DataFrame (
    {
        "Parameter" : ['# Azimuth Elements', 'Azimuth Beamwidth',
                       '# TX Elevation Elements', 'TX Elevation Beamwidth', 
                       'TX Gain', 'Ground Swath'],
        "Value" : [azNumElements, Utils.rad2deg(radarsat2_F0W3_TxAnt.azimuth_beamwidth(Wavelength)),
                   numTxElevElements, Utils.rad2deg(radarsat2_F0W3_TxAnt.elevation_beamwidth(Wavelength)),
                   10*np.log10(radarsat2_F0W3_TxAnt.gain(Wavelength)),
                   txGroundSwath /1000],
        "Unit" : ['','deg', 
                  '','deg', 
                  'dBi', 'km']
    }
)
titles = ['Parameter', 'Value', 'Unit']
df.reindex(columns=titles)

### RX Antenna Properties

In [ ]:
# initialise the RX antenna object
radarsat2_COM_RxAnt = PAA.PlanarArrayAntenna(num_el_elems    = elNumElements, 
                                             num_az_elems    = azNumElements, 
                                             el_elem_spacing = desiredElvElemSpacing, 
                                             az_elem_spacing = desiredAzElemSpacing, 
                                             efficiency      = efficiency)

# get the ground swath covered by the receive beam 
_, rxGroundSwath = SEM.SwathGroundSwathFromBeamwidth(Altitude,
                                                     Utils.deg2rad(AntennaOffset), 
                                                     radarsat2_COM_RxAnt.elevation_beamwidth(Wavelength))

In [ ]:
# display the RX antenna characteristics in a table format
pd.set_option("display.precision", 3)
df = pd.DataFrame (
    {
        "Parameter" : ['# Azimuth Elements', 'Azimuth Beamwidth',
                       '# RX Elevation Elements', 'RX Elevation Beamwidth', 
                       'RX Gain'],
        "Value" : [azNumElements, Utils.rad2deg(radarsat2_COM_RxAnt.azimuth_beamwidth(Wavelength)),
                   elNumElements, Utils.rad2deg(radarsat2_COM_RxAnt.elevation_beamwidth(Wavelength)),
                   10*np.log10(radarsat2_COM_RxAnt.gain(Wavelength))],
        "Unit" : ['','deg', 
                  '','deg', 'dBi']
    }
)
titles = ['Parameter', 'Value', 'Unit']
df.reindex(columns=titles)

### Antenna Patterns

In [ ]:
# antenna pattern for TX and RX antennas
AntennaPatternRange = np.arange(-90, 90, 0.1)

# Azimuth angles used only to visualise the antenna pattern in azimuth
azimuthAngles = np.arange(-5, 5, 0.01)   # azimuth angles in degrees

# ---------------------------------------------------
# TX antenna power patterns
# beam 1
txElvPat1 = radarsat2_F0W1_TxAnt.elevation_power_pattern(Wavelength, Utils.deg2rad(AntennaPatternRange), 0, 1, True)
txAzPat1 = radarsat2_F0W1_TxAnt.azimuth_power_pattern(Wavelength, Utils.deg2rad(azimuthAngles), 0, 1, True)
# beam 2
txElvPat2 = radarsat2_F0W2_TxAnt.elevation_power_pattern(Wavelength, Utils.deg2rad(AntennaPatternRange), 0, 1, True)
txAzPat2 = radarsat2_F0W2_TxAnt.azimuth_power_pattern(Wavelength, Utils.deg2rad(azimuthAngles), 0, 1, True)
# beam 3
txElvPat3 = radarsat2_F0W3_TxAnt.elevation_power_pattern(Wavelength, Utils.deg2rad(AntennaPatternRange), 0, 1, True)
txAzPat3 = radarsat2_F0W3_TxAnt.azimuth_power_pattern(Wavelength, Utils.deg2rad(azimuthAngles), 0, 1, True)

# RX antenna power patterns
rxElvPat = radarsat2_COM_RxAnt.elevation_power_pattern(Wavelength, Utils.deg2rad(AntennaPatternRange), 0, 1, True)
rxAzPat = radarsat2_COM_RxAnt.azimuth_power_pattern(Wavelength, Utils.deg2rad(azimuthAngles), 0, 1, True)

# TX / RX antenna gain scaling factor
gainScale1 = radarsat2_F0W1_TxAnt.gain(Wavelength) / radarsat2_COM_RxAnt.gain(Wavelength)
gainScale2 = radarsat2_F0W2_TxAnt.gain(Wavelength) / radarsat2_COM_RxAnt.gain(Wavelength)
gainScale3 = radarsat2_F0W3_TxAnt.gain(Wavelength) / radarsat2_COM_RxAnt.gain(Wavelength)


# ---------------------------------------------------
# Plot the antenna patterns
fig, (ax1, ax2) = plt.subplots(2,1,figsize = (12,6))
# TX patterns
ax1.plot(AntennaPatternRange, 10*np.log10(txElvPat1 * gainScale1), color="red", label=r"F0W1")
ax1.plot(AntennaPatternRange, 10*np.log10(txElvPat2 * gainScale2), color="green", label=r"F0W2")
ax1.plot(AntennaPatternRange, 10*np.log10(txElvPat3 * gainScale3), color="tab:orange", label=r"F0W3")
# rx pattern
ax1.plot(AntennaPatternRange, 10*np.log10(rxElvPat), color="black", linestyle="dashed", label=r"RX")

ax1.legend(loc=2)
ax1.set_title(r"Normalised Planar Array Elevation Antenna Pattern, $G_{el}(\theta)$ vs Angle")
ax1.set_xlabel(r"Angle from Antenna Center (deg)")
ax1.set_ylabel(r"Antenna pattern, $G_{el}(\theta)$ (dB)")
ax1.grid(linestyle='dotted')
ax1.set_ylim(-40, 5)
ax1.set_xlim(-25, 25)

# azimuth patterns
# TX patterns
ax2.plot(azimuthAngles, 10*np.log10(txAzPat1 * gainScale1), color="red", label=r"F0W1")
ax2.plot(azimuthAngles, 10*np.log10(txAzPat2 * gainScale2), color="green", label=r"F0W2")
ax2.plot(azimuthAngles, 10*np.log10(txAzPat3 * gainScale3), color="tab:orange", label=r"F0W3")
# Rx patterns
ax2.plot(azimuthAngles, 10*np.log10(rxAzPat), color="black", linestyle="dashed", label=r"RX")

ax2.legend(loc=2)
ax2.set_title(r"Normalised Planar Array Azimuth Power Pattern, $G_{az}(\theta)$ vs Angle")
ax2.set_xlabel(r"Angle from Antenna Center (deg)")
ax2.set_ylabel(r"Antenna pattern, $G_{az}(\theta)$ (dB)")
ax2.grid(linestyle='dotted')
ax2.set_ylim(-40, 5)
ax2.set_xlim(azimuthAngles.min(), azimuthAngles.max())

plt.tight_layout()
plt.show()

## PRF Analysis (RX beam)

### PRF vs Altitude

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 30
    numNadirEchoes = 11
    
    # define the swath width (in meters) ranges to use 
    altitudes = np.arange(600e3, 905e3, 5e3)
    
    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, altitudes)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(altitudes)
    
    # get the mid-swath look angle
    lookAngle = SEM.LookAngle(altitudes, MidSwathIncidenceAngle)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(rxGroundSwath,
                                             altitudes, 
                                             MidSwathIncidenceAngle,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    # plot the Nadir limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(altitudes/1000, nadirLower, color="red", alpha=0.6)
        ax1.plot(altitudes/1000, nadirUpper, color="red", alpha=0.6)
        plt.fill_between(altitudes/1000, nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(altitudes/1000, prfLower, color="black", alpha=0.6)
        ax1.plot(altitudes/1000, prfUpper, color="black", alpha=0.6)
        plt.fill_between(altitudes/1000, prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot PRF min
    ax1.plot(altitudes/1000, prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.plot(altitudes/1000, prfMax, color='tab:orange', linestyle='dashed') 
    # plot the altitude and ideal PRF
    ax1.axhline(y=Prf, color="black", linestyle="dashed")
    ax1.axvline(x=Altitude/1000, color="black", linestyle="dashed")
    
    #ax1.set_title('PRF vs Altitude')
    ax1.set_xlabel(r'Altitude (km)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(2200, 3000)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_10a_aesa_verification_02_prf_vs_alt.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_10a_aesa_verification_02_prf_vs_alt.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_10a_aesa_verification_02_prf_vs_alt.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

### PRF vs Antenna Length

In [ ]:
# define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses   = 32
    numNadirEchoes = 12
    
    # define the antenna length range to cover
    antennaLength = np.arange(5, 20.1, 0.1)
    
    # get the minimum PRF
    prfMin = []
    for ant in (antennaLength):
        prfMin.append(PRF.PrfMin(ant, Altitude))
    prfMin = np.asarray(prfMin)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # get the mid-swath look angle
    lookAngle = SEM.LookAngle(Altitude, MidSwathIncidenceAngle)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(rxGroundSwath,
                                             Altitude, 
                                             MidSwathIncidenceAngle,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    print("Nadir delay:            {:.6f} s".format(nadirDelay))
    print("Near range Delay (tn):  {:.6f} s".format(nearDelay))
    print("Far range Delay (tf):   {:.6f} s".format(farDelay))
    print("tf - tn:                {:.6f} s".format(farDelay - nearDelay))
    print("PRF max:                {:.6f} Hz".format(prfMax))  
    print("PRF min ({}m antenna): {:.6f} Hz".format(antLength, PRF.PrfMin(antLength, Altitude)))  
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.axhline(y=nadirLower, color="red", alpha=0.6)
        ax1.axhline(y=nadirUpper, color="red", alpha=0.6)
        plt.fill_between(antennaLength, nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.axhline(y=prfLower, color="black", alpha=0.6)
        ax1.axhline(y=prfUpper, color="black", alpha=0.6)
        plt.fill_between(antennaLength, prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot PRF min
    ax1.plot(antennaLength, prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.axhline(y=prfMax, color='tab:orange', linestyle='dashed') 
    # plot antenna length and ideal PRF
    ax1.axhline(y=Prf, color="black", linestyle='dashed') 
    ax1.axvline(x=antLength, color='black', linestyle='dashed') 
    
    
    #ax1.set_title('PRF vs Antenna length')
    ax1.set_xlabel(r'Antenna length (m)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(2200, 3000)
    plt.grid(linestyle='dotted')
    ax1.margins(0)
    plt.tight_layout()
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_10b_aesa_verification_02_prf_vs_ant.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_10b_aesa_verification_02_prf_vs_ant.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_10b_aesa_verification_02_prf_vs_ant.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

### PRF vs Pulse Length

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 30
    numNadirEchoes = 11
    
    # define the pulse length range to cover
    pulseLengths = np.arange(0, 101e-6, 1e-6)
    
    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, Altitude)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # get the mid-swath look angle
    lookAngle = SEM.LookAngle(Altitude, MidSwathIncidenceAngle)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(rxGroundSwath,
                                             Altitude, 
                                             MidSwathIncidenceAngle,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             pulseLengths)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       pulseLengths,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(pulseLengths, (farDelay - nearDelay))
    
    # display/plot the results
    print("Near range delay: {:.6f} s".format(nearDelay))
    print("Far range delay:  {:.6f} s".format(farDelay))
    print("Nadir delay:      {:.6f} s".format(nadirDelay))
    print("Minimum PRF:      {:.2f} Hz".format(prfMin))
    
    # plot PRF vs pulse length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(pulseLengths* 1e6, nadirLower, color="red", alpha=0.6)
        ax1.plot(pulseLengths* 1e6, nadirUpper, color="red", alpha=0.6)
        plt.fill_between(pulseLengths* 1e6, nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(pulseLengths* 1e6, prfLower, color="black", alpha=0.6)
        ax1.plot(pulseLengths* 1e6, prfUpper, color="black", alpha=0.6)
        plt.fill_between(pulseLengths* 1e6, prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot PRF min
    ax1.axhline(y=prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.plot(pulseLengths* 1e6, prfMax, color='tab:orange', linestyle='dashed')  
    # plot pulse length and ideal PRF
    ax1.axvline(x=PulseLen*1e6, color='black', linestyle='dashed')  
    ax1.axhline(y=Prf, color='black', linestyle='dashed')  
    
    ax1.set_title('PRF vs Pulse Lengh')
    ax1.set_xlabel(r'Pulse Lengh, $\tau_p$ ($\mu$s)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(2200, 2700)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_10c_aesa_verification_02_prf_vs_pul.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_10c_aesa_verification_02_prf_vs_pul.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_10c_aesa_verification_02_prf_vs_pul.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

### PRF vs Swath Width

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 30
    numNadirEchoes = 11
    
    # define the swath width (in meters) ranges to use 
    groundSwaths = np.arange(5e3, 51e3, 1e3)
    
    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, Altitude)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # get the mid-swath look angle
    lookAngle = Utils.deg2rad(AntennaOffset)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(groundSwaths,
                                             Altitude, 
                                             MidSwathIncidenceAngle,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    print("Nadir delay: {:.6f} s".format(nadirDelay))
    print("Minimum PRF: {:.2f} Hz".format(prfMin))
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(groundSwaths/1000, nadirLower, color="red", alpha=0.6)
        ax1.plot(groundSwaths/1000, nadirUpper, color="red", alpha=0.6)
        plt.fill_between(groundSwaths/1000, nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(groundSwaths/1000, prfLower, color="black", alpha=0.6)
        ax1.plot(groundSwaths/1000, prfUpper, color="black", alpha=0.6)
        plt.fill_between(groundSwaths/1000, prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot PRF min
    ax1.axhline(y=prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.plot(groundSwaths/1000, prfMax, color='tab:orange', linestyle='dashed')     
    
    # plot ideal ground swath and prf
    ax1.axhline(y=Prf, color="black", linestyle="dashed")
    ax1.axvline(x=rxGroundSwath/1000, color="black", linestyle="dashed")
    
    #ax1.set_title('PRF vs Ground Swath')
    ax1.set_xlabel(r'Ground swath, $W_{rg}$ (km)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(2200, 2700)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_10d_aesa_verification_02_prf_vs_swa.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_10d_aesa_verification_02_prf_vs_swa.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_10d_aesa_verification_02_prf_vs_swa.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

### PRF vs Incidence Angle

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 41
    numNadirEchoes = 25
    
    # define the incidence angle (in degreees) ranges to use 
    incidenceAngles = Utils.deg2rad(np.arange(15, 71, 1))

    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, Altitude)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # get the mid-swath look angle
    lookAngle = SEM.LookAngle(Altitude, incidenceAngles)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(rxGroundSwath,
                                             Altitude, 
                                             incidenceAngles,
                                             lookAngle)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    print("Nadir delay: {:.6f} s".format(nadirDelay))
    print("Minimum PRF: {:.2f} Hz".format(prfMin))
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(Utils.rad2deg(incidenceAngles), prfLower, color="black", alpha=0.6)
        ax1.plot(Utils.rad2deg(incidenceAngles), prfUpper, color="black", alpha=0.6)
        plt.fill_between(Utils.rad2deg(incidenceAngles), prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(Utils.rad2deg(incidenceAngles), nadirLower, color="red", alpha=0.6)
        ax1.plot(Utils.rad2deg(incidenceAngles), nadirUpper, color="red", alpha=0.6)
        plt.fill_between(Utils.rad2deg(incidenceAngles), nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot PRF min
    ax1.axhline(y=prfMin, color='tab:green', linestyle='dashed')    
    # plot PRFM Maximum
    ax1.plot(Utils.rad2deg(incidenceAngles), prfMax, color='tab:orange', linestyle='dashed')     
    # plot the incidence angle and ideal PRF
    ax1.axvline(x=Utils.rad2deg(MidSwathIncidenceAngle), color="black", linestyle="dashed")
    ax1.axhline(y=Prf, color="black", linestyle="dashed")
    
    #ax1.set_title('PRF vs Incidence Angle')
    ax1.set_xlabel(r'Incidence Angle, $\theta_{i,m}$ (deg)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(2200, 2700)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_10e_aesa_verification_02_prf_vs_inc.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_10e_aesa_verification_02_prf_vs_inc.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_10e_aesa_verification_02_prf_vs_inc.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

### PRF vs Look Angle

In [ ]:
if is_main_module():
    # define the number of PRF pulse and Nadir echoes to work with
    numPrfPulses = 53
    numNadirEchoes = 36
    
    # define the look angle (in degreees) ranges to use 
    lookAngles = LookAngleRangeRad
    
    # calculate the incidence angles from these look angles
    incidenceAngles = IncidenceAngleRangeRad

    # get the minimum PRF
    prfMin = PRF.PrfMin(antLength, Altitude)
    
    # get the nadir delay
    nadirDelay = PRF.NadirDelay(Altitude)
    
    # calculate the near/far swath slant ranges
    nearSR, farSR = SEM.NearAndFarSlantRanges(rxGroundSwath,
                                             Altitude, 
                                             incidenceAngles,
                                             lookAngles)
    
    # calculate the near/far range round trip delays
    nearDelay = SEM.RoundTripTime(nearSR)
    farDelay  = SEM.RoundTripTime(farSR)
    
    # get the PRF blind ranges
    prfBlindL, prfBlindU = PRF.PrfBlindRanges(numPrfPulses,
                                             nearDelay,
                                             farDelay,
                                             PulseLen)
    
    # get the nadir echo ranges
    nearNE, farNE = PRF.NadirEchoRanges(numNadirEchoes,
                                       nearDelay,
                                       farDelay,
                                       PulseLen,
                                       nadirDelay)
       
    # get the maximum PRF
    prfMax = PRF.PrfMax(PulseLen, (farDelay - nearDelay))
    
    # display/plot the results
    print("Nadir delay: {:.2f} us".format(nadirDelay * 1e6))
    print("Minimum PRF: {:.2f} Hz".format(prfMin))
    
    # plot PRF vs antenna length
    fig, ax1 = plt.subplots(1,1, figsize = (6,4))   
    
    # plot the PRF limits
    for prfLower, prfUpper in zip(prfBlindL[1:], prfBlindU):
        ax1.plot(Utils.rad2deg(lookAngles), prfLower, color="black", alpha=0.6)
        ax1.plot(Utils.rad2deg(lookAngles), prfUpper, color="black", alpha=0.6)
        plt.fill_between(Utils.rad2deg(lookAngles), prfLower, prfUpper, facecolor='blue', alpha=0.2)    
    # plot the Nadi limits
    for nadirLower, nadirUpper in zip(nearNE[1:], farNE):
        ax1.plot(Utils.rad2deg(lookAngles), nadirLower, color="red", alpha=0.6)
        ax1.plot(Utils.rad2deg(lookAngles), nadirUpper, color="red", alpha=0.6)
        plt.fill_between(Utils.rad2deg(lookAngles), nadirLower, nadirUpper, facecolor='yellow', alpha=0.2)    
    # plot PRF min
    ax1.axhline(y=prfMin, color='tab:green', linestyle='dashed')    
    # plot PRF Maximum
    ax1.plot(Utils.rad2deg(lookAngles), prfMax, color='tab:orange', linestyle='dashed')   
    # plot antenna mounting angle and ideal prf
    ax1.axvline(x=AntennaOffset, color="black", linestyle="dashed")
    ax1.axhline(y=Prf, color="black", linestyle="dashed")
    
    #ax1.set_title('PRF vs Look Angle')
    ax1.set_xlabel(r'Look Angle, $\gamma_m$ (deg)')
    ax1.set_ylabel(r'PRF (Hz)')    
    ax1.set_ylim(2200, 2700)
#     ax1.set_xlim(13, 46)
    plt.grid(linestyle='dotted')
    plt.tight_layout()
    ax1.margins(0)
    
    if SAVE_FIGURES:
        if SAVE_FIGURE_FORMAT == "SVG":
            plt.savefig('../figures/fig_10f_aesa_verification_02_prf_vs_loo.svg', format='svg', bbox_inches='tight')  
        elif SAVE_FIGURE_FORMAT == "PDF":
            plt.savefig('../figures/fig_10f_aesa_verification_02_prf_vs_loo.pdf', format='pdf', bbox_inches='tight')  
        else:
            plt.savefig('../figures/fig_10f_aesa_verification_02_prf_vs_loo.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
    else:
        plt.show()

## Performance

### NESZ

In [ ]:
# Calculate the antenna patterns used for calculating the NESZ
NumSamples = 100
NumRxScanAngles = 11
NumTxScanAngles = 1

# Near beam
txAnt = radarsat2_F0W1_TxAnt
rxAnt = radarsat2_COM_RxAnt

RxAntPatterns_F0W1, TxAntPatterns_F0W1, rxtxAnglesRad_F0W1, rxPeaks_F0W1, RxAntPatMaxAngles_F0W1, RxAntPatMaxValues_F0W1 \
    = NESZ.CalNeszAntennaPatterns_alt(rxAnt, txAnt, Wavelength, NumRxScanAngles, NumTxScanAngles, 
                                  LookAngleRangeRad.min(), LookAngleRangeRad.max(), NumSamples)

# Middle beam
txAnt = radarsat2_F0W2_TxAnt
rxAnt = radarsat2_COM_RxAnt

RxAntPatterns_F0W2, TxAntPatterns_F0W2, rxtxAnglesRad_F0W2, rxPeaks_F0W2, RxAntPatMaxAngles_F0W2, RxAntPatMaxValues_F0W2 \
    = NESZ.CalNeszAntennaPatterns_alt(rxAnt, txAnt, Wavelength, NumRxScanAngles, NumTxScanAngles, 
                                  LookAngleRangeRad.min(), LookAngleRangeRad.max(), NumSamples)

# Far beam
txAnt = radarsat2_F0W3_TxAnt
rxAnt = radarsat2_COM_RxAnt

RxAntPatterns_F0W3, TxAntPatterns_F0W3, rxtxAnglesRad_F0W3, rxPeaks_F0W3, RxAntPatMaxAngles_F0W3, RxAntPatMaxValues_F0W3 \
    = NESZ.CalNeszAntennaPatterns_alt(rxAnt, txAnt, Wavelength, NumRxScanAngles, NumTxScanAngles, 
                                  LookAngleRangeRad.min(), LookAngleRangeRad.max(), NumSamples)

# -----------------------------------------------
if DEBUG_PRINT_PLOTS:
    # Plot the antenna patterns
    fig, (ax1, ax2, ax3) = plt.subplots(3,1,figsize = (12,10))
    for i in range(NumRxScanAngles * NumTxScanAngles):
        if i == 0:
            # near beam
            ax1.plot(Utils.rad2deg(rxtxAnglesRad_F0W1[i]) + near_beam_look_angle, 10*np.log10(TxAntPatterns_F0W1[i]), color="red", label=r"F0W1")
            ax1.plot(Utils.rad2deg(rxtxAnglesRad_F0W1[i]) + near_beam_look_angle, 10*np.log10(RxAntPatterns_F0W1[i]), color="black", linestyle="dashdot", label=r"RX Elevation Pattern")
            # middle beam
            ax2.plot(Utils.rad2deg(rxtxAnglesRad_F0W2[i]) + middle_beam_look_angle, 10*np.log10(TxAntPatterns_F0W2[i]), color="green", label=r"F0W2")
            ax2.plot(Utils.rad2deg(rxtxAnglesRad_F0W2[i]) + middle_beam_look_angle, 10*np.log10(RxAntPatterns_F0W2[i]), color="black", linestyle="dashdot", label=r"RX Elevation Pattern")
            # far beam
            ax3.plot(Utils.rad2deg(rxtxAnglesRad_F0W3[i]) + far_beam_look_angle, 10*np.log10(TxAntPatterns_F0W3[i]), color="tab:orange", label=r"F0W3")
            ax3.plot(Utils.rad2deg(rxtxAnglesRad_F0W3[i]) + far_beam_look_angle, 10*np.log10(RxAntPatterns_F0W3[i]), color="black", linestyle="dashdot", label=r"RX Elevation Pattern")
        else:
            # near beam
            ax1.plot(Utils.rad2deg(rxtxAnglesRad_F0W1[i]) + near_beam_look_angle, 10*np.log10(TxAntPatterns_F0W1[i]), color="red")
            ax1.plot(Utils.rad2deg(rxtxAnglesRad_F0W1[i]) + near_beam_look_angle, 10*np.log10(RxAntPatterns_F0W1[i]), color="black", linestyle="dashdot")
            # middle beam
            ax2.plot(Utils.rad2deg(rxtxAnglesRad_F0W2[i]) + middle_beam_look_angle, 10*np.log10(TxAntPatterns_F0W2[i]), color="green")
            ax2.plot(Utils.rad2deg(rxtxAnglesRad_F0W2[i]) + middle_beam_look_angle, 10*np.log10(RxAntPatterns_F0W2[i]), color="black", linestyle="dashdot")
            # far beam
            ax3.plot(Utils.rad2deg(rxtxAnglesRad_F0W3[i]) + far_beam_look_angle, 10*np.log10(TxAntPatterns_F0W3[i]), color="tab:orange")
            ax3.plot(Utils.rad2deg(rxtxAnglesRad_F0W3[i]) + far_beam_look_angle, 10*np.log10(RxAntPatterns_F0W3[i]), color="black", linestyle="dashdot")

    # near beam
    ax1.plot(Utils.rad2deg(RxAntPatMaxAngles_F0W1/1000)  + near_beam_look_angle, 10*np.log10(RxAntPatMaxValues_F0W1), color="blue", label=r"Effective RX Elevation Pattern")
    # middle beam
    ax2.plot(Utils.rad2deg(RxAntPatMaxAngles_F0W2/1000)  + middle_beam_look_angle, 10*np.log10(RxAntPatMaxValues_F0W2), color="blue", label=r"Effective RX Elevation Pattern")
    # far beam
    ax3.plot(Utils.rad2deg(RxAntPatMaxAngles_F0W3/1000)  + far_beam_look_angle, 10*np.log10(RxAntPatMaxValues_F0W3), color="blue", label=r"Effective RX Elevation Pattern")

    ax1.set_title("F0W1 3dB Antenna pattern vs Look Angle")
    ax1.set_xlabel(r"Look Angle (deg)")
    ax1.set_ylabel(r"Antenna pattern (dB)")
    ax1.grid(linestyle='dotted')
    ax1.legend()

    ax2.set_title("F0W2 3dB Antenna pattern vs Look Angle")
    ax2.set_xlabel(r"Look Angle (deg)")
    ax2.set_ylabel(r"Antenna pattern (dB)")
    ax2.grid(linestyle='dotted')
    ax2.legend()

    ax3.set_title("F0W3 3dB Antenna pattern vs Look Angle")
    ax3.set_xlabel(r"Look Angle (deg)")
    ax3.set_ylabel(r"Antenna pattern (dB)")
    ax3.grid(linestyle='dotted')
    ax3.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# calculate the incidence angles and slant ranges over the antenna beam pattern
# near beam
IA_Arr_F0W1, SR_Arr_F0W1 = NESZ.CalcAnglesAndSlantRanges(NumRxScanAngles * NumTxScanAngles, Altitude, rxtxAnglesRad_F0W1, Utils.deg2rad(near_beam_look_angle))
# middle beam
IA_Arr_F0W2, SR_Arr_F0W2 = NESZ.CalcAnglesAndSlantRanges(NumRxScanAngles * NumTxScanAngles, Altitude, rxtxAnglesRad_F0W2, Utils.deg2rad(middle_beam_look_angle))
# far beam
IA_Arr_F0W3, SR_Arr_F0W3 = NESZ.CalcAnglesAndSlantRanges(NumRxScanAngles * NumTxScanAngles, Altitude, rxtxAnglesRad_F0W3, Utils.deg2rad(far_beam_look_angle))

In [ ]:
# calculate the core angles subtended by the slant ranges, and from these the ground ranges
# near beam
GR_Arr_F0W1 = NESZ.CalcGroundRange(NumRxScanAngles * NumTxScanAngles, Altitude, SR_Arr_F0W1)
# middle beam
GR_Arr_F0W2 = NESZ.CalcGroundRange(NumRxScanAngles * NumTxScanAngles, Altitude, SR_Arr_F0W2)
# far beam
GR_Arr_F0W3 = NESZ.CalcGroundRange(NumRxScanAngles * NumTxScanAngles, Altitude, SR_Arr_F0W3)

In [ ]:
# Calulate the value of NESZ of the look angles
# near beam
NESZ_Arr_F0W1, Nesz_max_incidence_angles_F0W1, Nesz_max_values_F0W1 \
    = NESZ.CalcNesz(NumRxScanAngles * NumTxScanAngles, 
    Altitude, 
    ChirpBandwidth, 
    IA_Arr_F0W1, SR_Arr_F0W1, 
    TxAntPatterns_F0W1, RxAntPatterns_F0W1, 
    Wavelength, TxPowerPeak, 
    PulseLen, Prf)
    
Nesz_dB_Arr_F0W1 = 10*np.log10(NESZ_Arr_F0W1)

# middle beam
NESZ_Arr_F0W2, Nesz_max_incidence_angles_F0W2, Nesz_max_values_F0W2 \
    = NESZ.CalcNesz(NumRxScanAngles * NumTxScanAngles, 
    Altitude, 
    ChirpBandwidth, 
    IA_Arr_F0W2, SR_Arr_F0W2, 
    TxAntPatterns_F0W2, RxAntPatterns_F0W2, 
    Wavelength, TxPowerPeak, 
    PulseLen, Prf)
    
Nesz_dB_Arr_F0W2 = 10*np.log10(NESZ_Arr_F0W2)

# far beam
NESZ_Arr_F0W3, Nesz_max_incidence_angles_F0W3, Nesz_max_values_F0W3 \
    = NESZ.CalcNesz(NumRxScanAngles * NumTxScanAngles, 
    Altitude, 
    ChirpBandwidth, 
    IA_Arr_F0W3, SR_Arr_F0W3, 
    TxAntPatterns_F0W3, RxAntPatterns_F0W3, 
    Wavelength, TxPowerPeak, 
    PulseLen, Prf)
    
Nesz_dB_Arr_F0W3 = 10*np.log10(NESZ_Arr_F0W3)

In [ ]:
# plot the NESZ
fig, ax1 = plt.subplots(figsize = (8, 4))
# secondary axis for ground range values
ax2 = ax1.twiny()
# Add some extra space for the second axis at the bottom
fig.subplots_adjust(bottom=0.2)

# locations on the first axis, where the second axis values are calculated
new_tick_locations = np.array([10, 15, 20., 25, 30., 35, 40., 45, 50., 55, 60])

# convert between the one axis to the next
def tick_function(altitude, x):
    V = SEM.IncidenceAngleToGroundRange(altitude, Utils.deg2rad(x))
    return ["%.0f" % z for z in V / 1000]

# Move twinned axis ticks and label from top to bottom
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.2))
# sett the ticks values and locations on the second axis
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
ax2.set_xlabel("Ground Range (km)")

for i in range(NumRxScanAngles * NumTxScanAngles):
    if i == 0:
        # near beam
        ax1.plot(Utils.rad2deg(IA_Arr_F0W1[i]), Nesz_dB_Arr_F0W1[i], color="red", ls="--", label="F0W1")
        # middle beam
        ax1.plot(Utils.rad2deg(IA_Arr_F0W2[i]), Nesz_dB_Arr_F0W2[i], color="green", ls="--", label="F0W2")
        # far beam
        ax1.plot(Utils.rad2deg(IA_Arr_F0W3[i]), Nesz_dB_Arr_F0W3[i], color="tab:orange", ls="--", label="F0W3")
    else:
        # near beam
        ax1.plot(Utils.rad2deg(IA_Arr_F0W1[i]), Nesz_dB_Arr_F0W1[i], color="red", ls="--")
        # middle beam
        ax1.plot(Utils.rad2deg(IA_Arr_F0W2[i]), Nesz_dB_Arr_F0W2[i], color="green", ls="--")
        # far beam
        ax1.plot(Utils.rad2deg(IA_Arr_F0W3[i]), Nesz_dB_Arr_F0W3[i], color="tab:orange", ls="--")
        
# plot required NESZ value +- 2dB
ax1.axhline(ReqNesz, color="blue", label="Required NESZ")
ax1.fill_between(ax1.get_xlim(), ReqNesz-2, ReqNesz+2, color="yellow", alpha=0.3)
# ax1.fill_between(ax1.get_xlim(), ReqNesz+2, -20, color="green", alpha=0.15)

# ax1.set_title("NESZ vs Incidence Angle")
ax1.set_xlabel(r"Incidence Angle (deg)")
ax1.set_ylabel(r"NESZ (dB)")
ax1.grid(linestyle='dotted')
ax1.set_ylim(-28, -15)
ax1.legend()
ax1.margins(0)
ax2.set_xlim(ax1.get_xlim())

plt.tight_layout()

if SAVE_FIGURES:
    if SAVE_FIGURE_FORMAT == "SVG":
        plt.savefig('../figures/fig_11_aesa_verification_02_nesz.svg', format='svg', bbox_inches='tight')  
    elif SAVE_FIGURE_FORMAT == "PDF":
        plt.savefig('../figures/fig_11_aesa_verification_02_nesz.pdf', format='pdf', bbox_inches='tight')  
    else:
        plt.savefig('../figures/fig_11_aesa_verification_02_nesz.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
else:
    plt.show()

### AASR

In [ ]:
# Calculate the antenna patterns for AASR calculationns
NumAmbiguities = 10
FrequencySteps = 100

# Get the processed Doppler bandwidth
DopplerBandwidth = SEM.ProcessedDopplerBandwidth(SEM.PlatformVelocity(Altitude), ReqResolutionAz)
print("Processed Doppler Bandwidth: {:.2f} Hz".format(DopplerBandwidth))

# DopplerFrequencies = np.linspace(-DopplerBandwidth/2, DopplerBandwidth/2, FrequencySteps)
DopplerFrequencies = np.linspace(start=-10000, stop=10000, num=1000)

AzimuthAngles = SEM.AzimuthAngleFromDoppler(DopplerFrequencies, Wavelength, SEM.PlatformVelocity(Altitude))
                        
# calculate the TX/RX antenna patterns over the Doppler frequency bandwidth
txAzPat_F0W1 = radarsat2_F0W1_TxAnt.azimuth_field_pattern(Wavelength, AzimuthAngles, 0, 1, True)
txAzPat_F0W2 = radarsat2_F0W2_TxAnt.azimuth_field_pattern(Wavelength, AzimuthAngles, 0, 1, True)
txAzPat_F0W3 = radarsat2_F0W3_TxAnt.azimuth_field_pattern(Wavelength, AzimuthAngles, 0, 1, True)
rxAzPat = radarsat2_COM_RxAnt.azimuth_field_pattern(Wavelength, AzimuthAngles, 0, 1, True)

txAzPowerPat_F0W1 = radarsat2_F0W1_TxAnt.azimuth_power_pattern(Wavelength, AzimuthAngles, 0, 1, True)
txAzPowerPat_F0W2 = radarsat2_F0W2_TxAnt.azimuth_power_pattern(Wavelength, AzimuthAngles, 0, 1, True)
txAzPowerPat_F0W3 = radarsat2_F0W3_TxAnt.azimuth_power_pattern(Wavelength, AzimuthAngles, 0, 1, True)
rxAzPowerPat = radarsat2_COM_RxAnt.azimuth_power_pattern(Wavelength, AzimuthAngles, 0, 1, True)

In [ ]:
# Plot the antenna patterns
fig, ax1 = plt.subplots(figsize = (12,3))
# TX beams
ax1.plot(Utils.rad2deg(AzimuthAngles), 10*np.log10(txAzPowerPat_F0W1 * gainScale1), color="red", label=r"F0W1")
ax1.plot(Utils.rad2deg(AzimuthAngles), 10*np.log10(txAzPowerPat_F0W2 * gainScale2), color="green", label=r"F0W2")
ax1.plot(Utils.rad2deg(AzimuthAngles), 10*np.log10(txAzPowerPat_F0W3 * gainScale3), color="tab:orange", label=r"F0W3")
# RX beam
ax1.plot(Utils.rad2deg(AzimuthAngles), 10*np.log10(rxAzPowerPat), color="black", label=r"RX", linestyle="dashed")

ax1.set_title("Noramlised 3dB Antenna azimuth power pattern vs Angle")
ax1.set_xlabel(r"Angle (deg)")
ax1.set_ylabel(r"Antenna pattern (dB)")
ax1.grid(linestyle='dotted')
ax1.set_ylim(-40, 5)
ax1.set_xlim(-1, 1)
ax1.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the AASR values
# near beam
Aasr_Arr_F0W1 = AASR.Get_AASR(DopplerBandwidth = DopplerBandwidth,
                     Frequencies = DopplerFrequencies,
                     NumAmbiguities = NumAmbiguities,
                     FreqSteps = FrequencySteps,
                     Prf = Prf,
                     TxAntennaPattern = txAzPat_F0W1,
                     RxAntennaPattern = rxAzPat,
                     TxGain = radarsat2_F0W1_TxAnt.gain(Wavelength),
                     RxGain = radarsat2_COM_RxAnt.gain(Wavelength) )
# middle beam
Aasr_Arr_F0W2 = AASR.Get_AASR(DopplerBandwidth = DopplerBandwidth,
                     Frequencies = DopplerFrequencies,
                     NumAmbiguities = NumAmbiguities,
                     FreqSteps = FrequencySteps,
                     Prf = Prf,
                     TxAntennaPattern = txAzPat_F0W2,
                     RxAntennaPattern = rxAzPat,
                     TxGain = radarsat2_F0W2_TxAnt.gain(Wavelength),
                     RxGain = radarsat2_COM_RxAnt.gain(Wavelength) )
# far beam
Aasr_Arr_F0W3 = AASR.Get_AASR(DopplerBandwidth = DopplerBandwidth,
                     Frequencies = DopplerFrequencies,
                     NumAmbiguities = NumAmbiguities,
                     FreqSteps = FrequencySteps,
                     Prf = Prf,
                     TxAntennaPattern = txAzPat_F0W3,
                     RxAntennaPattern = rxAzPat,
                     TxGain = radarsat2_F0W3_TxAnt.gain(Wavelength),
                     RxGain = radarsat2_COM_RxAnt.gain(Wavelength) )

print("AASR_F0W1: {:.3f} dB".format(10*np.log10(Aasr_Arr_F0W1)))
print("AASR_F0W2: {:.3f} dB".format(10*np.log10(Aasr_Arr_F0W2)))
print("AASR_F0W3: {:.3f} dB".format(10*np.log10(Aasr_Arr_F0W3)))

# create dummy ground swath array over which to plot the AASR data
# SwathArr_F0W1 = np.linspace(np.asarray(SR_Arr_F0W1).min(), np.asarray(SR_Arr_F0W1).max(), 50)
# SwathArr_F0W2 = np.linspace(np.asarray(SR_Arr_F0W2).min(), np.asarray(SR_Arr_F0W2).max(), 50)
# SwathArr_F0W3 = np.linspace(np.asarray(SR_Arr_F0W3).min(), np.asarray(SR_Arr_F0W3).max(), 50)
SwathArr_F0W1 = np.linspace(np.asarray(IA_Arr_F0W1).min(), np.asarray(IA_Arr_F0W1).max(), 50)
SwathArr_F0W2 = np.linspace(np.asarray(IA_Arr_F0W2).min(), np.asarray(IA_Arr_F0W2).max(), 50)
SwathArr_F0W3 = np.linspace(np.asarray(IA_Arr_F0W3).min(), np.asarray(IA_Arr_F0W3).max(), 50)

# create an array equal to the ground swath array and fill with the AASR data
Aasr_Arr_Full_F0W1 = np.full((SwathArr_F0W1.shape[0]), 10*np.log10(Aasr_Arr_F0W1))
Aasr_Arr_Full_F0W2 = np.full((SwathArr_F0W2.shape[0]), 10*np.log10(Aasr_Arr_F0W2))
Aasr_Arr_Full_F0W3 = np.full((SwathArr_F0W3.shape[0]), 10*np.log10(Aasr_Arr_F0W3))

In [ ]:
# plot the AASR vs ground swath
fig, ax1 = plt.subplots(1,1, figsize = (8,4))   

# secondary axis for ground range values
ax2 = ax1.twiny()
# Add some extra space for the second axis at the bottom
fig.subplots_adjust(bottom=0.2)

# locations on the first axis, where the second axis values are calculated
new_tick_locations = np.array([10, 15, 20., 25, 30., 35, 40., 45, 50., 55, 60])

# convert between the one axis to the next
def tick_function(altitude, x):
    V = SEM.IncidenceAngleToGroundRange(altitude, Utils.deg2rad(x))
    return ["%.0f" % z for z in V / 1000]

# Move twinned axis ticks and label from top to bottom
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.2))
# sett the ticks values and locations on the second axis
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
ax2.set_xlabel("Ground Range (km)")

ax1.plot(np.rad2deg(SwathArr_F0W1), Aasr_Arr_Full_F0W1, color='red', label=r'F0W1 AASR with $B_\gamma$ = {:.2f} Hz'.format(DopplerBandwidth))  
ax1.plot(np.rad2deg(SwathArr_F0W2), Aasr_Arr_Full_F0W2, color='green', label=r'F0W2 AASR with $B_\gamma$ = {:.2f} Hz'.format(DopplerBandwidth))  
ax1.plot(np.rad2deg(SwathArr_F0W3), Aasr_Arr_Full_F0W3, color='tab:orange', label=r'F0W3 AASR with $B_\gamma$ = {:.2f} Hz'.format(DopplerBandwidth))  
ax1.axhline(y=ReqAasr, label="Required AASR", color="blue")

# ax1.set_title('Azimuth-Ambiguity-to-Signal Ratio vs Slant Range')
ax1.set_ylabel(r'AASR (dB)')
ax1.set_xlabel(r'Incidence Angle (deg)')    
plt.grid(linestyle='dotted')
ax1.set_ylim(-60, -17)
plt.tight_layout()
ax1.legend()
ax1.margins(0)

ax2.set_xlim(ax1.get_xlim())

if SAVE_FIGURES:
    if SAVE_FIGURE_FORMAT == "SVG":
        plt.savefig('../figures/fig_12_aesa_verification_02_aasr.svg', format='svg', bbox_inches='tight')  
    elif SAVE_FIGURE_FORMAT == "PDF":
        plt.savefig('../figures/fig_12_aesa_verification_02_aasr.pdf', format='pdf', bbox_inches='tight')  
    else:
        plt.savefig('../figures/fig_12_aesa_verification_02_aasr.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
else:
    plt.show()

### RASR

In [ ]:
# Calculate the RASR for a number of PRFs
NF = 5
NN = 3
NumSamples = 100
NumRxScanAngles = 31

# ----------------------------------------------------------
# calculate the TX/RX antenna patterns over the main 3dB TX beamwidth
txBW_F0W1 = radarsat2_F0W1_TxAnt.elevation_beamwidth(Wavelength)
txBW_F0W2 = radarsat2_F0W2_TxAnt.elevation_beamwidth(Wavelength)
txBW_F0W3 = radarsat2_F0W3_TxAnt.elevation_beamwidth(Wavelength)

# receiver beamwidth
rxBW = radarsat2_COM_RxAnt.elevation_beamwidth(Wavelength)

In [ ]:
# Main 3dB beam RASR values
# Get the scan angle increment. The -1 is to ensure there is a overlap at the 2 beam peaks
rxAngleInc_F0W1 = txBW_F0W1 / (NumRxScanAngles -1) 
rxAngleInc_F0W2 = txBW_F0W2 / (NumRxScanAngles -1) 
rxAngleInc_F0W3 = txBW_F0W3 / (NumRxScanAngles -1) 

RxAntPatterns_F0W1 = []
TxAntPatterns_F0W1 = []
rxtxAnglesRad_F0W1 = []

RxAntPatterns_F0W2 = []
TxAntPatterns_F0W2 = []
rxtxAnglesRad_F0W2 = []

RxAntPatterns_F0W3 = []
TxAntPatterns_F0W3 = []
rxtxAnglesRad_F0W3 = []

# ----------------------------------------------------------
# main beam properties
for i in range(NumRxScanAngles):
    # near beam
    anglesRad_F0W1 = np.linspace((-(txBW_F0W1/2) + i*rxAngleInc_F0W1) - (rxBW/2), 
                            (-(txBW_F0W1/2) + i*rxAngleInc_F0W1) + (rxBW/2), 
                            NumSamples)
    txPat_F0W1 = radarsat2_F0W1_TxAnt.elevation_field_pattern(Wavelength, anglesRad_F0W1, 0, 1, True) * radarsat2_F0W1_TxAnt.gain(Wavelength)
    rxPat_F0W1 = radarsat2_COM_RxAnt.elevation_field_pattern(Wavelength, 
                                                             anglesRad_F0W1, 
                                                             (-(txBW_F0W1/2) + i*rxAngleInc_F0W1),
                                                             1, True) * radarsat2_COM_RxAnt.gain(Wavelength)
    rxtxAnglesRad_F0W1.append(anglesRad_F0W1)
    TxAntPatterns_F0W1.append(txPat_F0W1)
    RxAntPatterns_F0W1.append(rxPat_F0W1)
    # middle beam
    anglesRad_F0W2 = np.linspace((-(txBW_F0W2/2) + i*rxAngleInc_F0W2) - (rxBW/2), 
                            (-(txBW_F0W2/2) + i*rxAngleInc_F0W2) + (rxBW/2), 
                            NumSamples)
    txPat_F0W2 = radarsat2_F0W2_TxAnt.elevation_field_pattern(Wavelength, anglesRad_F0W2, 0, 1, True) * radarsat2_F0W2_TxAnt.gain(Wavelength)
    rxPat_F0W2 = radarsat2_COM_RxAnt.elevation_field_pattern(Wavelength, 
                                                             anglesRad_F0W2, 
                                                             (-(txBW_F0W2/2) + i*rxAngleInc_F0W2),
                                                             1, True) * radarsat2_COM_RxAnt.gain(Wavelength)
    rxtxAnglesRad_F0W2.append(anglesRad_F0W2)
    TxAntPatterns_F0W2.append(txPat_F0W2)
    RxAntPatterns_F0W2.append(rxPat_F0W2)
    # far beam
    anglesRad_F0W3 = np.linspace((-(txBW_F0W3/2) + i*rxAngleInc_F0W3) - (rxBW/2), 
                            (-(txBW_F0W3/2) + i*rxAngleInc_F0W3) + (rxBW/2), 
                            NumSamples)
    txPat_F0W3 = radarsat2_F0W3_TxAnt.elevation_field_pattern(Wavelength, anglesRad_F0W3, 0, 1, True) * radarsat2_F0W3_TxAnt.gain(Wavelength)
    rxPat_F0W3 = radarsat2_COM_RxAnt.elevation_field_pattern(Wavelength, 
                                                             anglesRad_F0W3, 
                                                             (-(txBW_F0W3/2) + i*rxAngleInc_F0W3),
                                                             1, True) * radarsat2_COM_RxAnt.gain(Wavelength)
    rxtxAnglesRad_F0W3.append(anglesRad_F0W3)
    TxAntPatterns_F0W3.append(txPat_F0W3)
    RxAntPatterns_F0W3.append(rxPat_F0W3)

# ----------------------------------------------------------
# Calculate the incidence anges and slant ranges over this rx/tx angle range of the 3dB main beam
# near beam
IA_Arr_F0W1 = []
SR_Arr_F0W1 = []
IA_Arr_F0W2 = []
SR_Arr_F0W2 = []
IA_Arr_F0W3 = []
SR_Arr_F0W3 = []
for i in range(NumRxScanAngles):
    # near beam
    ia_arr_F0W1 = SEM.IncidenceAngle(Altitude, rxtxAnglesRad_F0W1[i] + Utils.deg2rad(near_beam_look_angle))
    sr_arr_F0W1 = SEM.SlantRange(Altitude, ia_arr_F0W1, rxtxAnglesRad_F0W1[i] + Utils.deg2rad(near_beam_look_angle))
    IA_Arr_F0W1.append(ia_arr_F0W1)
    SR_Arr_F0W1.append(sr_arr_F0W1)
    # middle beam
    ia_arr_F0W2 = SEM.IncidenceAngle(Altitude, rxtxAnglesRad_F0W2[i] + Utils.deg2rad(middle_beam_look_angle))
    sr_arr_F0W2 = SEM.SlantRange(Altitude, ia_arr_F0W2, rxtxAnglesRad_F0W2[i] + Utils.deg2rad(middle_beam_look_angle))
    IA_Arr_F0W2.append(ia_arr_F0W2)
    SR_Arr_F0W2.append(sr_arr_F0W2)
    # far beam
    ia_arr_F0W3 = SEM.IncidenceAngle(Altitude, rxtxAnglesRad_F0W3[i] + Utils.deg2rad(far_beam_look_angle))
    sr_arr_F0W3 = SEM.SlantRange(Altitude, ia_arr_F0W3, rxtxAnglesRad_F0W3[i] + Utils.deg2rad(far_beam_look_angle))
    IA_Arr_F0W3.append(ia_arr_F0W3)
    SR_Arr_F0W3.append(sr_arr_F0W3)

In [ ]:
# Plot the main antenna patterns
fig, ax1 = plt.subplots(figsize = (8,4))
for i in range(NumRxScanAngles):
    # bear beam
    ax1.plot(Utils.rad2deg(rxtxAnglesRad_F0W1[i]), 10*np.log10(TxAntPatterns_F0W1[i]), color="red")
    ax1.plot(Utils.rad2deg(rxtxAnglesRad_F0W1[i]), 10*np.log10(RxAntPatterns_F0W1[i]), color="black", linestyle="dashed")
    # middle beam
    ax1.plot(Utils.rad2deg(rxtxAnglesRad_F0W2[i]), 10*np.log10(TxAntPatterns_F0W2[i]), color="green")
    ax1.plot(Utils.rad2deg(rxtxAnglesRad_F0W2[i]), 10*np.log10(RxAntPatterns_F0W2[i]), color="black", linestyle="dashed")
    # far beam
    ax1.plot(Utils.rad2deg(rxtxAnglesRad_F0W3[i]), 10*np.log10(TxAntPatterns_F0W3[i]), color="tab:orange")
    ax1.plot(Utils.rad2deg(rxtxAnglesRad_F0W3[i]), 10*np.log10(RxAntPatterns_F0W3[i]), color="black", linestyle="dashed")
    
ax1.set_title("3dB Antenna pattern vs Angle")
ax1.set_xlabel(r"Angle (deg)")
ax1.set_ylabel(r"Antenna pattern (dB)")
ax1.grid(linestyle='dotted')
ax1.legend()
plt.show()

In [ ]:
# RASR main values
RasrMain_Arr_F0W1 = []
RasrMain_Arr_F0W2 = []
RasrMain_Arr_F0W3 = []

for i in range(NumRxScanAngles):
    # near beam
    RasrMain_F0W1 = RASR.Get_RasrMain(SlantRange=SR_Arr_F0W1[i],
                                IncidenceAngle = IA_Arr_F0W1[i],
                                TxAntennaPattern = TxAntPatterns_F0W1[i], 
                                RxAntennaPattern = RxAntPatterns_F0W1[i] )
    RasrMain_Arr_F0W1.append(RasrMain_F0W1)
    # middle beam
    RasrMain_F0W2 = RASR.Get_RasrMain(SlantRange=SR_Arr_F0W2[i],
                                IncidenceAngle = IA_Arr_F0W2[i],
                                TxAntennaPattern = TxAntPatterns_F0W2[i], 
                                RxAntennaPattern = RxAntPatterns_F0W2[i] )
    RasrMain_Arr_F0W2.append(RasrMain_F0W2)
    # far beam
    RasrMain_F0W3 = RASR.Get_RasrMain(SlantRange=SR_Arr_F0W3[i],
                                IncidenceAngle = IA_Arr_F0W3[i],
                                TxAntennaPattern = TxAntPatterns_F0W3[i], 
                                RxAntennaPattern = RxAntPatterns_F0W3[i] )
    RasrMain_Arr_F0W3.append(RasrMain_F0W3)
    
RasrMain_Arr_dB_F0W1 = 10*np.log10(RasrMain_Arr_F0W1)
RasrMain_Arr_dB_F0W2 = 10*np.log10(RasrMain_Arr_F0W2)
RasrMain_Arr_dB_F0W3 = 10*np.log10(RasrMain_Arr_F0W3)

In [ ]:
# Plot RASR Main values
fig, ax1 = plt.subplots(figsize = (8, 6))
for i in range(NumRxScanAngles):
    if i == 0:
        ax1.plot(Utils.rad2deg(IA_Arr_F0W1[i]), RasrMain_Arr_dB_F0W1[i], color="red", ls="--", label="F0W1 RASR")
        ax1.plot(Utils.rad2deg(IA_Arr_F0W2[i]), RasrMain_Arr_dB_F0W2[i], color="green", ls="--", label="F0W2 RASR")
        ax1.plot(Utils.rad2deg(IA_Arr_F0W3[i]), RasrMain_Arr_dB_F0W3[i], color="tab:orange", ls="--", label="F0W3 RASR")
    else:
        ax1.plot(Utils.rad2deg(IA_Arr_F0W1[i]), RasrMain_Arr_dB_F0W1[i], color="red", ls="--")
        ax1.plot(Utils.rad2deg(IA_Arr_F0W2[i]), RasrMain_Arr_dB_F0W2[i], color="green", ls="--")
        ax1.plot(Utils.rad2deg(IA_Arr_F0W3[i]), RasrMain_Arr_dB_F0W3[i], color="tab:orange", ls="--")

ax1.axhline(ReqNesz, color="blue", label="Required RASR")
ax1.set_title("RASR vs Slant Range")
ax1.set_xlabel(r"Incidence Angle (deg)")
ax1.set_ylabel(r"RASR (dB)")
ax1.grid(linestyle='dotted')
ax1.legend()
plt.tight_layout()
plt.show()

In [ ]:
# RASR ambiguities
# ambiguity slant ranges and angles
SR_Amb_Scan_Arr_F0W1 = []
IA_Amb_Scan_Arr_F0W1 = []
TX_Amb_Scan_Arr_F0W1 = []
RX_Amb_Scan_Arr_F0W1 = []

SR_Amb_Scan_Arr_F0W2 = []
IA_Amb_Scan_Arr_F0W2 = []
TX_Amb_Scan_Arr_F0W2 = []
RX_Amb_Scan_Arr_F0W2 = []

SR_Amb_Scan_Arr_F0W3 = []
IA_Amb_Scan_Arr_F0W3 = []
TX_Amb_Scan_Arr_F0W3 = []
RX_Amb_Scan_Arr_F0W3 = []
for i in range(NumRxScanAngles):
    SR_Amb_Arr_F0W1 = []
    IA_Amb_Arr_F0W1 = []
    TX_Amb_Arr_F0W1 = []
    RX_Amb_Arr_F0W1 = []
    
    SR_Amb_Arr_F0W2 = []
    IA_Amb_Arr_F0W2 = []
    TX_Amb_Arr_F0W2 = []
    RX_Amb_Arr_F0W2 = []
    
    SR_Amb_Arr_F0W3 = []
    IA_Amb_Arr_F0W3 = []
    TX_Amb_Arr_F0W3 = []
    RX_Amb_Arr_F0W3 = []

    for j in range(-NN, NF+1, 1):
        if j != 0:
            # for each ambiguity, calculate the slant range the incidence angle and the look angle
            # then calculate the antenna pattern angle from this look angle, and calculate the 
            # antenna patterns
            # save all these values for the summation calculation
            # near beam
            sr_amb_arr_F0W1 = SR_Arr_F0W1[i] + (Constants.SPEED_OF_LIGHT/2)*(j/Prf)
            ia_amb_arr_F0W1 = SEM.IncidenceAngleFromSlantRange(Altitude, sr_amb_arr_F0W1)
            la_amb_arr_F0W1 = SEM.LookAngle(Altitude, ia_amb_arr_F0W1)
            ant_amb_angle_arr_F0W1 = la_amb_arr_F0W1 - Utils.deg2rad(near_beam_look_angle)
            txPatamb_F0W1 = radarsat2_F0W1_TxAnt.elevation_field_pattern(Wavelength, ant_amb_angle_arr_F0W1, 0, 1, True) * radarsat2_F0W1_TxAnt.gain(Wavelength)
            rxPatamb_F0W1 = radarsat2_COM_RxAnt.elevation_field_pattern(Wavelength, ant_amb_angle_arr_F0W1, 0, 1, True) * radarsat2_COM_RxAnt.gain(Wavelength)
            SR_Amb_Arr_F0W1.append(sr_amb_arr_F0W1)
            IA_Amb_Arr_F0W1.append(ia_amb_arr_F0W1)
            TX_Amb_Arr_F0W1.append(txPatamb_F0W1)
            RX_Amb_Arr_F0W1.append(rxPatamb_F0W1)
            
            # middle beam
            sr_amb_arr_F0W2 = SR_Arr_F0W2[i] + (Constants.SPEED_OF_LIGHT/2)*(j/Prf)
            ia_amb_arr_F0W2 = SEM.IncidenceAngleFromSlantRange(Altitude, sr_amb_arr_F0W2)
            la_amb_arr_F0W2 = SEM.LookAngle(Altitude, ia_amb_arr_F0W2)
            ant_amb_angle_arr_F0W2 = la_amb_arr_F0W2 - Utils.deg2rad(middle_beam_look_angle)
            txPatamb_F0W2 = radarsat2_F0W2_TxAnt.elevation_field_pattern(Wavelength, ant_amb_angle_arr_F0W2, 0, 1, True) * radarsat2_F0W2_TxAnt.gain(Wavelength)
            rxPatamb_F0W2 = radarsat2_COM_RxAnt.elevation_field_pattern(Wavelength, ant_amb_angle_arr_F0W2, 0, 1, True) * radarsat2_COM_RxAnt.gain(Wavelength)
            SR_Amb_Arr_F0W2.append(sr_amb_arr_F0W2)
            IA_Amb_Arr_F0W2.append(ia_amb_arr_F0W2)
            TX_Amb_Arr_F0W2.append(txPatamb_F0W2)
            RX_Amb_Arr_F0W2.append(rxPatamb_F0W2)
            
            # far beam
            sr_amb_arr_F0W3 = SR_Arr_F0W3[i] + (Constants.SPEED_OF_LIGHT/2)*(j/Prf)
            ia_amb_arr_F0W3 = SEM.IncidenceAngleFromSlantRange(Altitude, sr_amb_arr_F0W3)
            la_amb_arr_F0W3 = SEM.LookAngle(Altitude, ia_amb_arr_F0W3)
            ant_amb_angle_arr_F0W3 = la_amb_arr_F0W3 - Utils.deg2rad(far_beam_look_angle)
            txPatamb_F0W3 = radarsat2_F0W3_TxAnt.elevation_field_pattern(Wavelength, ant_amb_angle_arr_F0W3, 0, 1, True) * radarsat2_F0W3_TxAnt.gain(Wavelength)
            rxPatamb_F0W3 = radarsat2_COM_RxAnt.elevation_field_pattern(Wavelength, ant_amb_angle_arr_F0W3, 0, 1, True) * radarsat2_COM_RxAnt.gain(Wavelength)
            SR_Amb_Arr_F0W3.append(sr_amb_arr_F0W3)
            IA_Amb_Arr_F0W3.append(ia_amb_arr_F0W3)
            TX_Amb_Arr_F0W3.append(txPatamb_F0W3)
            RX_Amb_Arr_F0W3.append(rxPatamb_F0W3)
            
    # append to the  Scan arrays
    # near beam
    SR_Amb_Scan_Arr_F0W1.append(SR_Amb_Arr_F0W1)
    IA_Amb_Scan_Arr_F0W1.append(IA_Amb_Arr_F0W1)
    TX_Amb_Scan_Arr_F0W1.append(TX_Amb_Arr_F0W1)
    RX_Amb_Scan_Arr_F0W1.append(RX_Amb_Arr_F0W1)
    # middle beam
    SR_Amb_Scan_Arr_F0W2.append(SR_Amb_Arr_F0W2)
    IA_Amb_Scan_Arr_F0W2.append(IA_Amb_Arr_F0W2)
    TX_Amb_Scan_Arr_F0W2.append(TX_Amb_Arr_F0W2)
    RX_Amb_Scan_Arr_F0W2.append(RX_Amb_Arr_F0W2)
    # far beam
    SR_Amb_Scan_Arr_F0W3.append(SR_Amb_Arr_F0W3)
    IA_Amb_Scan_Arr_F0W3.append(IA_Amb_Arr_F0W3)
    TX_Amb_Scan_Arr_F0W3.append(TX_Amb_Arr_F0W3)
    RX_Amb_Scan_Arr_F0W3.append(RX_Amb_Arr_F0W3)

# convert lists to arrays
# near beam
SR_Amb_Scan_Arr_F0W1 = np.asarray(SR_Amb_Scan_Arr_F0W1)
IA_Amb_Scan_Arr_F0W1 = np.asarray(IA_Amb_Scan_Arr_F0W1)
TX_Amb_Scan_Arr_F0W1 = np.asarray(TX_Amb_Scan_Arr_F0W1)
RX_Amb_Scan_Arr_F0W1 = np.asarray(RX_Amb_Scan_Arr_F0W1)
# middle beam
SR_Amb_Scan_Arr_F0W2 = np.asarray(SR_Amb_Scan_Arr_F0W2)
IA_Amb_Scan_Arr_F0W2 = np.asarray(IA_Amb_Scan_Arr_F0W2)
TX_Amb_Scan_Arr_F0W2 = np.asarray(TX_Amb_Scan_Arr_F0W2)
RX_Amb_Scan_Arr_F0W2 = np.asarray(RX_Amb_Scan_Arr_F0W2)
# far beam
SR_Amb_Scan_Arr_F0W3 = np.asarray(SR_Amb_Scan_Arr_F0W3)
IA_Amb_Scan_Arr_F0W3 = np.asarray(IA_Amb_Scan_Arr_F0W3)
TX_Amb_Scan_Arr_F0W3 = np.asarray(TX_Amb_Scan_Arr_F0W3)
RX_Amb_Scan_Arr_F0W3 = np.asarray(RX_Amb_Scan_Arr_F0W3)

In [ ]:
# Calculate the RASR values for the ambiguous PRFs
def RasrAmb(TxPat, RxPat, SR, IA):
    """
    Calculate the ambiguity inner sum
    """
    antPat = np.power(np.multiply(TxPat, RxPat), 2)
    return antPat / (np.power(SR, 3) * np.sin(IA))

# for each slant range and each ambiguity count, calculate the ambiguous value
Rasr_Amb_Scan_Arr_F0W1 = []
Rasr_Amb_Scan_Arr_F0W2 = []
Rasr_Amb_Scan_Arr_F0W3= []

for i in range(NumRxScanAngles):
    # near beam
    Rasr_Amb_Arr_F0W1 = []
    for j in range(SR_Amb_Scan_Arr_F0W1[i].shape[0]):
        Rasr_Amb_Arr_F0W1.append(RasrAmb(TX_Amb_Scan_Arr_F0W1[i][j], RX_Amb_Scan_Arr_F0W1[i][j], SR_Amb_Scan_Arr_F0W1[i][j], IA_Amb_Scan_Arr_F0W1[i][j]))
    Rasr_Amb_Scan_Arr_F0W1.append(Rasr_Amb_Arr_F0W1)
    # middle beam
    Rasr_Amb_Arr_F0W2 = []
    for j in range(SR_Amb_Scan_Arr_F0W2[i].shape[0]):
        Rasr_Amb_Arr_F0W2.append(RasrAmb(TX_Amb_Scan_Arr_F0W2[i][j], RX_Amb_Scan_Arr_F0W2[i][j], SR_Amb_Scan_Arr_F0W2[i][j], IA_Amb_Scan_Arr_F0W2[i][j]))
    Rasr_Amb_Scan_Arr_F0W2.append(Rasr_Amb_Arr_F0W2)
    # far beam
    Rasr_Amb_Arr_F0W3 = []
    for j in range(SR_Amb_Scan_Arr_F0W1[i].shape[0]):
        Rasr_Amb_Arr_F0W3.append(RasrAmb(TX_Amb_Scan_Arr_F0W3[i][j], RX_Amb_Scan_Arr_F0W3[i][j], SR_Amb_Scan_Arr_F0W3[i][j], IA_Amb_Scan_Arr_F0W3[i][j]))
    Rasr_Amb_Scan_Arr_F0W3.append(Rasr_Amb_Arr_F0W3)
    
Rasr_Amb_Scan_Arr_F0W1 = np.asarray(Rasr_Amb_Scan_Arr_F0W1)
Rasr_Amb_Scan_Arr_F0W2 = np.asarray(Rasr_Amb_Scan_Arr_F0W2)
Rasr_Amb_Scan_Arr_F0W2 = np.asarray(Rasr_Amb_Scan_Arr_F0W3)

In [ ]:
# calculate the Total RASR value for each slant range

# add all the ambiguities for a given slant range together,
# this is the total ambiguity singal for a given slant range
Rasr_Amb_Scan_Sum_F0W1 = []
Rasr_Amb_Scan_Sum_F0W2 = []
Rasr_Amb_Scan_Sum_F0W3 = []

for i in range(NumRxScanAngles):
    Rasr_Amb_Scan_Sum_F0W1.append(np.nansum(Rasr_Amb_Scan_Arr_F0W1[i], axis=0))
    Rasr_Amb_Scan_Sum_F0W2.append(np.nansum(Rasr_Amb_Scan_Arr_F0W2[i], axis=0))
    Rasr_Amb_Scan_Sum_F0W3.append(np.nansum(Rasr_Amb_Scan_Arr_F0W3[i], axis=0))
    
Rasr_Amb_Scan_Sum_F0W1 = np.asarray(Rasr_Amb_Scan_Sum_F0W1)
Rasr_Amb_Scan_Sum_F0W2 = np.asarray(Rasr_Amb_Scan_Sum_F0W2)
Rasr_Amb_Scan_Sum_F0W3 = np.asarray(Rasr_Amb_Scan_Sum_F0W3)

# calculate the Total RASR value for each slant range
RASR_Total_Arr_F0W1 = []
RASR_Total_Arr_F0W2 = []
RASR_Total_Arr_F0W3 = []

for i in range(NumRxScanAngles):
    RASR_Arr_F0W1 = []
    RASR_Arr_F0W2 = []
    RASR_Arr_F0W3 = []
    for j in range(SR_Arr_F0W1[i].shape[0]):
        RASR_Arr_F0W1.append(RasrMain_Arr_F0W1[i][j] * Rasr_Amb_Scan_Sum_F0W1[i][j])
        RASR_Arr_F0W2.append(RasrMain_Arr_F0W2[i][j] * Rasr_Amb_Scan_Sum_F0W2[i][j])
        RASR_Arr_F0W3.append(RasrMain_Arr_F0W3[i][j] * Rasr_Amb_Scan_Sum_F0W3[i][j])
    
    RASR_Total_Arr_F0W1.append(RASR_Arr_F0W1)
    RASR_Total_Arr_F0W2.append(RASR_Arr_F0W2)
    RASR_Total_Arr_F0W3.append(RASR_Arr_F0W3)
    
RASR_Total_Arr_F0W1 = np.asarray(RASR_Total_Arr_F0W1)
RASR_Total_Arr_F0W2 = np.asarray(RASR_Total_Arr_F0W2)
RASR_Total_Arr_F0W3 = np.asarray(RASR_Total_Arr_F0W3)

RASR_Total_Arr_dB_F0W1 = 10*np.log10(RASR_Total_Arr_F0W1)
RASR_Total_Arr_dB_F0W2 = 10*np.log10(RASR_Total_Arr_F0W2)
RASR_Total_Arr_dB_F0W3 = 10*np.log10(RASR_Total_Arr_F0W3)

In [ ]:
# Plot RASR values vs ground
fig, ax1 = plt.subplots(figsize = (8,4))

# secondary axis for ground range values
ax2 = ax1.twiny()
# Add some extra space for the second axis at the bottom
fig.subplots_adjust(bottom=0.2)

# locations on the first axis, where the second axis values are calculated
new_tick_locations = np.array([15, 20., 25., 30., 35., 40., 45.])

# convert between the one axis to the next
def tick_function(altitude, x):
    V = SEM.IncidenceAngleToGroundRange(altitude, Utils.deg2rad(x))
    return ["%.0f" % z for z in V / 1000]

# Move twinned axis ticks and label from top to bottom
ax2.xaxis.set_ticks_position("bottom")
ax2.xaxis.set_label_position("bottom")

# Offset the twin axis below the host
ax2.spines["bottom"].set_position(("axes", -0.2))
# sett the ticks values and locations on the second axis
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(tick_function(Altitude, new_tick_locations))
ax2.set_xlabel("Ground Range (km)")

for i in range(NumRxScanAngles):
    if i == 0:
        ax1.plot(Utils.rad2deg(IA_Arr_F0W1[i]), RASR_Total_Arr_dB_F0W1[i], color="red", ls="--", label="F0W1 RASR")
        ax1.plot(Utils.rad2deg(IA_Arr_F0W2[i]), RASR_Total_Arr_dB_F0W2[i], color="green", ls="--", label="F0W1 RASR")
        ax1.plot(Utils.rad2deg(IA_Arr_F0W3[i]), RASR_Total_Arr_dB_F0W3[i], color="tab:orange", ls="--", label="F0W1 RASR")
    else:
        ax1.plot(Utils.rad2deg(IA_Arr_F0W1[i]), RASR_Total_Arr_dB_F0W1[i], color="red", ls="--")
        ax1.plot(Utils.rad2deg(IA_Arr_F0W2[i]), RASR_Total_Arr_dB_F0W2[i], color="green", ls="--")
        ax1.plot(Utils.rad2deg(IA_Arr_F0W3[i]), RASR_Total_Arr_dB_F0W3[i], color="tab:orange", ls="--")
        
ax1.axhline(ReqNesz, color="blue", label="Required RASR")

# ax1.set_title("RASR vs Slant Range")
ax1.set_xlabel(r"Incidence Angle (deg)")
ax1.set_ylabel(r"RASR (dB)")
ax1.grid(linestyle='dotted')
ax1.legend()
plt.tight_layout()
ax1.margins(0)
ax2.set_xlim(ax1.get_xlim())

if SAVE_FIGURES:
    if SAVE_FIGURE_FORMAT == "SVG":
        plt.savefig('../figures/fig_13_aesa_verification_02_rasr.svg', format='svg', bbox_inches='tight')  
    elif SAVE_FIGURE_FORMAT == "PDF":
        plt.savefig('../figures/fig_13_aesa_verification_02_rasr.pdf', format='pdf', bbox_inches='tight')  
    else:
        plt.savefig('../figures/fig_13_aesa_verification_02_rasr.png', format='png', bbox_inches='tight', dpi=SAVE_FIGURE_DPI)  
else:
    plt.show()